In [1]:
from src.sentinel_api import get_sentinel_api
from src.constant import GEOJSON_PATH, DOWNLOADED_DATA_PATH, REMOVE_ZIPS
import os

# get api 
api = get_sentinel_api()

In [2]:
# search by polygon, time, and Hub query keywords
products_query = api.query(
                    limit = 1,
                    filename = 'S3A_*',
                    producttype = 'SY_2_SYN___')

os.makedirs(DOWNLOADED_DATA_PATH, exist_ok = True) 
products = api.download_all(products_query, DOWNLOADED_DATA_PATH)
print(products)

Downloading: 100%|██████████| 927M/927M [00:37<00:00, 24.9MB/s]
MD5 checksumming: 100%|██████████| 927M/927M [00:29<00:00, 31.2MB/s](OrderedDict([('a980fb05-e058-4757-9daf-43dd36a92579', {'id': 'a980fb05-e058-4757-9daf-43dd36a92579', 'title': 'S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002', 'size': 926579072, 'md5': '3C5743583ACE93D095C6BDB49232F6E0', 'date': datetime.datetime(2020, 5, 30, 5, 17, 40, 785000), 'footprint': 'POLYGON((65.8114 41.9773,66.6439 41.9,67.4689 41.8177,68.2877 41.73,69.1144 41.6353,69.9412 41.5344,70.7722 41.4266,71.5626 41.3183,72.3709 41.2012,73.176 41.0785,73.9904 40.9532,74.7863 40.8196,75.5839 40.6825,76.3741 40.5352,77.1589 40.386,77.9438 40.2281,78.7297 40.0677,79.5072 39.902,80.2831 39.7306,81.0582 39.5532,82.106 42.1516,83.2388 44.7439,84.4691 47.3262,85.8156 49.8941,84.8967 50.101,83.9796 50.2984,83.0607 50.4873,82.1303 50.6687,81.1926 50.8466,80.2453 51.0146,79.2941 51.1774,78.3339 51.3275,77.3662 51.47

In [3]:
file_titles = []
zip_files = []

for product in products:
    for product_id in product:
        file_titles.append(product[product_id]['title'])
        zip_files.append(product[product_id]['path'])

        import zipfile
        archive = zipfile.ZipFile(product[product_id]['path'], 'r')
        archive.extractall(path=DOWNLOADED_DATA_PATH)

for zip_file in zip_files:
    if REMOVE_ZIPS:
        os.remove(zip_file)    

In [4]:
from netCDF4 import Dataset
for file_title in file_titles:
    data_path = os.path.join(DOWNLOADED_DATA_PATH, '{}.SEN3'.format(file_title))
    for nc_file_name in sorted(os.listdir(data_path)):
        if nc_file_name.endswith('.nc'):
            nc_file_path =  os.path.join(data_path, nc_file_name)  
            print(nc_file_path)
            rootgrp = Dataset(nc_file_path, "r", format="NETCDF4")

            for i in rootgrp.variables:
                print(i, rootgrp.variables[i].shape)

            print('\n')

./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_AMIN.nc
AMIN (4091, 4865)


./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_AOT550.nc
T550 (4091, 4865)
T550_err (4091, 4865)


./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_Angstrom_exp550.nc
A550 (4091, 4865)


./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_Oa01_reflectance.nc
SDR_Oa01 (4091, 4865)
SDR_Oa01_err (4091, 4865)


./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_Oa02_reflectance.nc
SDR_Oa02 (4091, 4865)
SDR_Oa02_err (4091, 4865)


./src/data/S3A_SY_2_SYN____20200530T051741_20200530T052041_20200531T164906_0179_059_005_2160_LN2_O_NT_002.SEN3/Syn_Oa03_reflectance.nc
SDR_Oa03 (4091, 4865)
SDR_Oa03_